In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1618583134697_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, r.review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2020-01-01' \
and contains(r.observations, 'FL') \
and review_length < 40 \
limit 2 \
"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           2 non-null      int64 
 1   REVIEW_HEADLINE  2 non-null      object
 2   REVIEW_COMMENTS  2 non-null      object
 3   LANG             2 non-null      object
 4   REVIEW_LENGTH    2 non-null      int64 
 5   FL               2 non-null      int64 
dtypes: int64(3), object(3)
memory usage: 224.0+ bytes

In [18]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#EXAMPLE
comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
text = "Excelente excelente"

print('Calling DetectDominantLanguage')
print(json.dumps(comprehend.detect_dominant_language(Text = text)))
print("End of DetectDominantLanguage\n")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Calling DetectDominantLanguage
{"Languages": [{"LanguageCode": "en", "Score": 0.5294792056083679}, {"LanguageCode": "es", "Score": 0.268625408411026}, {"LanguageCode": "pt", "Score": 0.20022545754909515}], "ResponseMetadata": {"RequestId": "a20c52da-2be3-487d-8332-98fdd7419e1a", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "a20c52da-2be3-487d-8332-98fdd7419e1a", "content-type": "application/x-amz-json-1.1", "content-length": "162", "date": "Thu, 08 Apr 2021 16:53:13 GMT"}, "RetryAttempts": 0}}
End of DetectDominantLanguage

In [11]:
comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
text = "Excelente excelente"

test=comprehend.detect_dominant_language(Text = text)['Languages'][0]['LanguageCode']
test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'en'

In [12]:
test2=comprehend.detect_dominant_language(Text = text)['Languages'][0]['Score']
test2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5294792056083679

In [14]:
test3=comprehend.detect_dominant_language(Text = text)
test3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'Languages': [{'LanguageCode': 'en', 'Score': 0.5294792056083679}, {'LanguageCode': 'es', 'Score': 0.268625408411026}, {'LanguageCode': 'pt', 'Score': 0.20022545754909515}], 'ResponseMetadata': {'RequestId': '5890b37d-f01b-4576-931b-0a750318e8da', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5890b37d-f01b-4576-931b-0a750318e8da', 'content-type': 'application/x-amz-json-1.1', 'content-length': '162', 'date': 'Fri, 16 Apr 2021 19:55:27 GMT'}, 'RetryAttempts': 0}}

In [15]:
test3['Languages'][0]['Score']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5294792056083679

In [19]:
comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['COMPREHEND'] = reviews['ALL_TEXT'].apply(lambda x: comprehend.detect_dominant_language(Text=x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID               REVIEW_HEADLINE                REVIEW_COMMENTS  \
0  314684049  사이즈는 크게 사시오                   실 사이즈보다 작다\n한치수 크게 사야함          
1  264758873  Une idée du prix sans fixe ?  Une idée du prix sans fixes ?   

  LANG  REVIEW_LENGTH  FL  \
0  en   21             1    
1  en   29             1    

                                                     ALL_TEXT  \
0  사이즈는 크게 사시오 실 사이즈보다 작다\n한치수 크게 사야함                           
1  Une idée du prix sans fixe ? Une idée du prix sans fixes ?   

                                                                                                                                                                                                                                                                                                                                                                               COMPREHEND  
0  {'Languages': [{'LanguageCode': 'ko', 'Score': 0.7575757503509521}], 'ResponseMetadata': {'RequestId'

In [99]:
#reviews['PRED_LANG'] = reviews['COMPREHEND'].apply(lambda x: x[0])

#feature3 = [d.get('Feature3') for d in df.dic]
reviews['COMPREHEND2'] = [d.get('Languages') for d in reviews['COMPREHEND']]
reviews['COMPREHEND3'] = reviews['COMPREHEND2'].str[0]
reviews['PRED_LANG'] = [d.get('LanguageCode') for d in reviews['COMPREHEND3']]
reviews['PROB'] = [d.get('Score') for d in reviews['COMPREHEND3']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
reviews

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID               REVIEW_HEADLINE                REVIEW_COMMENTS  \
0  314684049  사이즈는 크게 사시오                   실 사이즈보다 작다\n한치수 크게 사야함          
1  264758873  Une idée du prix sans fixe ?  Une idée du prix sans fixes ?   

  LANG  REVIEW_LENGTH  FL  \
0  en   21             1    
1  en   29             1    

                                                     ALL_TEXT  \
0  사이즈는 크게 사시오 실 사이즈보다 작다\n한치수 크게 사야함                           
1  Une idée du prix sans fixe ? Une idée du prix sans fixes ?   

                                                                                                                                                                                                                                                                                                                                                                               COMPREHEND  \
0  {'Languages': [{'LanguageCode': 'ko', 'Score': 0.7575757503509521}], 'ResponseMetadata': {'RequestId

In [48]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0

#'ca' is catalan which is close to spanish
reviews.loc[ (reviews['LANG']=='es') & (reviews['PRED_LANG']=='ca') , 'model_FL'] = 0

reviews.loc[ (reviews['LANG']=='zh') & (reviews['PRED_LANG']=='zh-CN') , 'model_FL'] = 0

#reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0

#reviews.loc[ (reviews['LANG']=='en') & ((reviews['firstword_eng']==True) | (reviews['secondword_eng']==True)) , 'model_FL'] = 0

#reviews.loc[reviews['PRED_LANG'] == 'UNKNOWN' , 'model_FL'] = 0

#reviews.loc[reviews['PRED_LANG'] == 'af' , 'model_FL'] = 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [51]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

93

In [52]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

907

In [53]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [54]:
FN.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        UGC_ID                                              REVIEW_HEADLINE  \
12   293203743  Very comfortable and muy buen choes..💯❤                       
14   307566627  Yes                                                           
17   275365325  ممتاز انصح به                                                 
25   312085548  Excelente!                                                    
30   312205024  Yes                                                           
31   314245602  I would buy this product again and again                      
32   283954418  si                                                            
59   284531360  www.dillards.com  1-800-DILLARD (800-345-5273)  Love it, Ha   
62   324847043  Yes                                                           
63   325854683  Hecho in China                                                
67   274098105  No                                                            
71   276243457  I would buy this product again and a

In [67]:
#Compare to google trans
from google_trans_new import google_translator  

reviews['PRED_LANG_GOOG'] = reviews['ALL_TEXT'].apply(lambda x: google_translator().detect(x)[0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

429 (Too Many Requests) from TTS API. Probable cause: Unknown
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/core/series.py", line 4138, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2467, in pandas._libs.lib.map_infer
  File "<stdin>", line 4, in <lambda>
  File "/usr/local/lib/python3.7/site-packages/google_trans_new/google_trans_new.py", line 247, in detect
    raise google_new_transError(tts=self, response=r)
google_trans_new.google_trans_new.google_new_transError: 429 (Too Many Requests) from TTS API. Probable cause: Unknown



In [ ]:
reviews.head()

In [27]:
reviews['model_FL_GOOG'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL_GOOG'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL_GOOG'] = 0

#'ca' is catalan which is close to spanish
reviews.loc[ (reviews['LANG']=='es') & (reviews['PRED_LANG']=='ca') , 'model_FL_GOOG'] = 0

reviews.loc[ (reviews['LANG']=='zh') & (reviews['PRED_LANG']=='zh-CN') , 'model_FL_GOOG'] = 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                                          REVIEW_HEADLINE  \
0  312249489  Si                                                        
1  340236839  Atención al cliente excelente                             
2  318873458  이게..뭐랄까....                                               
3  311820670  Esta bici esta dlv no sirve para atropellar embarazadas   
4  274919545  いい感じ                                                      

                REVIEW_COMMENTS LANG  REVIEW_LENGTH  FL  \
0  El micrófono es espectacular  en   28             1    
1  Buena calidad y durabilidad   en   27             1    
2  전에 벤지탈때쓰던 클립온바가 훨 편하다는게 함정    en   26             1    
3  Vale vrg la bici              en   16             1    
4  650B　本所フェンダー取付　最強です           en   19             1    

                                                                   ALL_TEXT  \
0  Si El micrófono es espectacular                                            
1  Atención al cliente excelente Buena calidad y

In [32]:
reviews = reviews.drop(['model_FL_GOOG'], axis =1 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("LANG", StringType(), True)\
,StructField("REVIEW_LENGTH", StringType(), True)\
,StructField("FL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("PRED_LANG", StringType(), True)\
,StructField("model_FL", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
reviews = spark.createDataFrame(reviews, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
reviews.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","FL_FL1_amazon_comprehend").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…